# **Create a Conversational ChtBot Using LangChain, OpenAI's Model and Various Prompting Techniques:**

In [1]:
# %pip install -qU langchain-openai

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

## **Load LLM & Embeddings:**

In [3]:
from langchain_openai import OpenAI, ChatOpenAI, OpenAIEmbeddings
import asyncio

In [4]:
# Load LLM:

def load_llm(model_name:str="gpt-4o", temperature: float=0.2, max_tokens:int=2000) -> OpenAI:
    llm = ChatOpenAI(model_name=model_name, temperature=temperature, max_tokens=max_tokens)
    return llm

llm = load_llm(model_name="gpt-4o", temperature=0.2, max_tokens=2000)

In [5]:
llm.invoke("HI")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 8, 'total_tokens': 18, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_17316073ab', 'id': 'chatcmpl-BMbiUFsXTIy57AJO2k5ePKYkHJYr7', 'finish_reason': 'stop', 'logprobs': None}, id='run-c7dd7962-e783-468d-be9c-399411557e4c-0', usage_metadata={'input_tokens': 8, 'output_tokens': 10, 'total_tokens': 18, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [6]:
# Load Embeddings:

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [7]:
len(embeddings.embed_query("Hello world"))

1536

## **Vector Store:**

In [8]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [9]:
# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

In [10]:
# retriever:

retriever = vectorstore.as_retriever(search_kwargs=dict(k=5), search_type="similarity", search_score=True, search_score_threshold=0.5)

In [11]:
retriever.invoke("What is the difference between LLMs and Agents?")

[Document(id='cd89a678-5e7c-4b71-9d67-3a562638ab86', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\nBuilding agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview#\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over

## **Build Conversational AI Bot:**

In [12]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, FewShotChatMessagePromptTemplate
from langchain.chains import create_history_aware_retriever
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.messages import HumanMessage, AIMessage

### **Step 1: Create History-Aware-Retriever:**

In [13]:
# Create History Aware Retriever:

retriever_prompt = (
    "Given a chat history and the latest user question which might reference context in the chat history,"
    "formulate a standalone question which can be understood without the chat history."
    "Do NOT answer the question, just reformulate it if needed and otherwise return it as is."
)


contextualize_q_prompt  = ChatPromptTemplate.from_messages(
    [
        ("system", retriever_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),


     ]
)

history_aware_retriever = create_history_aware_retriever(llm, retriever, contextualize_q_prompt)

### **Step 2: Create Question-Aware-Chain Using Prompting Techniques:**

1. System Prompt
2. Few-Shot Prompt
3. Chat-Prompt

In [14]:
# Write System Messages:
system_prompt = (
    "You are an AI Assistant named 'Lily,' specializing in AI and ML topics for students or users. "
    "This AI Assistant is designed by Dibyendu Biswas, a Generative AI Engineer. "
    "You provide clear, step-by-step explanations and encourage students to learn. "
    "If a question is unclear, ask for clarification. "
    "Use four sentences maximum and keep the answer concise. "
    "\n\n"
    "{context}"
)

# Define Few-Shot Prompt Template using Few Examples:
few_shot_examples = [
    {'input': 'Hello', 'output': 'Hi there! How can I help you today?'},
    {'input': 'What is your name?', 'output': 'I’m Lily, your assistant, specializing in AI and ML topics for students!'},
    {'input': 'How are you?', 'output': 'I’m doing great, thanks for asking! How about you?'},
    {'input': 'Tell me a joke', 'output': 'Why don’t skeletons fight each other? They don’t have the guts!'},
    {'input': 'What time is it?', 'output': 'I’m not able to tell the time, but you can check your device!'},
    {'input': 'Goodbye', 'output': 'Goodbye! Have a great day!'},
    {'input': 'Can you help me?', 'output': 'Of course! What do you need help with?'},
    {'input': 'What is the weather like?', 'output': 'I’m unable to check real-time weather, but you can look it up online!'},
    {'input': 'Set a reminder', 'output': 'I can’t set reminders, but I can help you with other things!'},
    {'input': 'Where are you from?', 'output': 'I don’t have a specific place, but I’m here to assist you wherever you are!'},
    {'input': 'What can you do?', 'output': 'I can answer questions, tell jokes, help with tasks, and more! What do you need today?'},
    {'input': 'Help me with math', 'output': 'Sure! What math problem would you like help with?'},
    {'input': 'I am bored', 'output': 'Let’s do something fun! Want to hear a joke or maybe try a quiz?'},
    {'input': 'What is 5 + 7?', 'output': '5 + 7 equals 12!'},
    {'input': 'How do I cook pasta?', 'output': 'Boil water, add pasta, cook for 8-12 minutes, drain, and enjoy!'},
    {'input': 'Tell me a fun fact', 'output': 'Did you know? A day on Venus is longer than a year on Venus!'},
    {'input': 'What is your favorite color?', 'output': 'I don’t have a favorite color, but I think blue is nice!'}
]

# Few Shot Template:
few_shot_template = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}")
    ]
)
# Few Shot Prompt:
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=few_shot_template,
    examples=few_shot_examples,
)

# Define Final Prompt Using System prompt and Few Shot Prompt
qa_prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    few_shot_prompt,
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])

In [15]:
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

### **Step 3: Create Rag-Chain using history_aware_retriever and question_answer_chain:**

In [16]:
# Create Rag-Chain using history_aware_retriever and question_answer_chain:

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

### **Step 4: Use the Memory and Session to store the current conversation:**

* ChatMessageHistory,
* BaseChatMessageHistory,
* RunnableWithMessageHistory

In [17]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory


#### **Create a Session to Store Conversation:**

In [18]:
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

#### **Create Conversational Rag Chain using `memory` and `rag_chain`:**

In [19]:
# conversational_rag_chain:

conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

## **Generate Response and store the current user session:**

In [20]:
from IPython.display import display, Markdown

In [21]:
ans = conversational_rag_chain.invoke(
    {"input": "What is Chain of Thought Prompt?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

display(Markdown(ans))

Chain of Thought (CoT) prompting is a technique used in large language models to enhance reasoning by breaking down a problem into a sequence of intermediate steps. This approach helps the model to process complex tasks more effectively by generating a series of logical steps or thoughts, leading to a final solution. It improves the model's ability to handle tasks that require multi-step reasoning.

In [22]:
ans= conversational_rag_chain.invoke(
    {"input": "What are common ways of doing it?"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

display(Markdown(ans))

Common ways to implement Chain of Thought prompting include: 

1. Simple prompting, such as asking the model to list steps for a task.
2. Using task-specific instructions, like providing a story outline for writing.
3. Incorporating human inputs to guide the decomposition of tasks into manageable steps.

In [23]:
store

{'abc123': InMemoryChatMessageHistory(messages=[HumanMessage(content='What is Chain of Thought Prompt?', additional_kwargs={}, response_metadata={}), AIMessage(content="Chain of Thought (CoT) prompting is a technique used in large language models to enhance reasoning by breaking down a problem into a sequence of intermediate steps. This approach helps the model to process complex tasks more effectively by generating a series of logical steps or thoughts, leading to a final solution. It improves the model's ability to handle tasks that require multi-step reasoning.", additional_kwargs={}, response_metadata={}), HumanMessage(content='What are common ways of doing it?', additional_kwargs={}, response_metadata={}), AIMessage(content='Common ways to implement Chain of Thought prompting include: \n\n1. Simple prompting, such as asking the model to list steps for a task.\n2. Using task-specific instructions, like providing a story outline for writing.\n3. Incorporating human inputs to guide t

In [26]:
ans = conversational_rag_chain.invoke(
    {"input": "My Name is Shyam and I am a student."},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

display(Markdown(ans))

Nice to meet you, Shyam! How can I assist you with your studies today?

In [27]:
ans = conversational_rag_chain.invoke(
    {"input": "What is my name?."},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

display(Markdown(ans))

Your name is Shyam!

In [28]:
for message in store["abc123"].messages:
    if isinstance(message, AIMessage):
        prefix = "AI"
    else:
        prefix = "User"

    print(f"{prefix}: {message.content}\n")

     

User: What is Chain of Thought Prompt?

AI: Chain of Thought (CoT) prompting is a technique used in large language models to enhance reasoning by breaking down a problem into a sequence of intermediate steps. This approach helps the model to process complex tasks more effectively by generating a series of logical steps or thoughts, leading to a final solution. It improves the model's ability to handle tasks that require multi-step reasoning.

User: What are common ways of doing it?

AI: Common ways to implement Chain of Thought prompting include: 

1. Simple prompting, such as asking the model to list steps for a task.
2. Using task-specific instructions, like providing a story outline for writing.
3. Incorporating human inputs to guide the decomposition of tasks into manageable steps.

User: What are the questions I asked before?

AI: I'm unable to recall previous interactions or questions. However, feel free to ask anything again!

User: My Name is Shyam and I am a student.

AI: Nice